In [3]:
import pandas as pd
import requests
from zipfile import ZipFile
from io import BytesIO
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import scipy.sparse

In [4]:
# Step 1: Load the Dataset Efficiently
url = "https://raw.githubusercontent.com/ftr-fybrlabs/HackUTD2024/main/data/current_customers.csv.zip"
r = requests.get(url)
z = ZipFile(BytesIO(r.content))
file_name = z.namelist()[0]
df = pd.read_csv(z.open(file_name))

C:\Users\Joshu\AppData\Local\Temp\ipykernel_2644\495329559.py:6: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(z.open(file_name))


In [5]:
# Step 2: Inspect and Drop Unnecessary Columns Early
print(df.head())

# Check the shape of the DataFrame
print(df.shape)

# Check data types of each column
print(df.dtypes)

# Check for missing values
print(df.isnull().sum())

                                             acct_id  extenders  \
0  00000950dea4a869e9fe70d823444d418c5abebbd8e830...          0   
1  00002ec815678c6b9837ad4c9db82ac7fe5fb6af5dc229...          1   
2  000050825287b713415614a237757da4cd7517365cbce2...          1   
3  000060be4aa292815abc44ab6fe96015b89e83b21c8a63...          0   
4  00008c39885815e42a0bb750cee199cd4da741a5645705...          1   

   wireless_clients_count  wired_clients_count    rx_avg_bps    tx_avg_bps  \
0                       3                    2  1.849822e+06  2.229501e+06   
1                      13                    6  5.316781e+05  3.170011e+05   
2                       9                    3  2.657035e+05  9.632467e+03   
3                       3                    3  1.077448e+06  4.149613e+05   
4                      19                    7  6.083613e+05  4.780259e+05   

     rx_p95_bps    tx_p95_bps    rx_max_bps    tx_max_bps  ...        city  \
0  2.652856e+06  2.243758e+06  6.606018e+06  2.390

In [6]:
# Step 3: Drop rows with missing values in the numerical columns early
numerical_cols = ['wireless_clients_count', 'wired_clients_count', 'rx_avg_bps', 'tx_avg_bps', 'rx_p95_bps', 'tx_p95_bps', 'rx_max_bps', 'tx_max_bps', 'rssi_mean', 'rssi_median', 'rssi_max', 'rssi_min']
df.dropna(subset=numerical_cols, inplace=True)


In [7]:
# Step 4: Convert Data Types for Efficiency
df[numerical_cols] = df[numerical_cols].astype(float)
categorical_cols = ['network_speed', 'city', 'state', 'whole_home_wifi', 'wifi_security', 'wifi_security_plus', 'premium_tech_pro', 'identity_protection', 'family_identity_protection', 'total_shield', 'youtube_tv']
df[categorical_cols] = df[categorical_cols].astype(str)

In [9]:
# Step 5: Normalizing the Numerical Data
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

In [10]:
#Step 6: Encoding the Categorical Data
encoder = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
encoded_sparse = encoder.fit_transform(df[categorical_cols])

In [11]:
# Step 7: Combine the Encoded Data with the Original DataFrame
encoded_df = pd.DataFrame.sparse.from_spmatrix(encoded_sparse, columns=encoder.get_feature_names_out(categorical_cols))
df = pd.concat([df, encoded_df], axis=1)

In [12]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 789689 entries, 0 to 789688
Columns: 1447 entries, acct_id to youtube_tv_nan
dtypes: Sparse[float64, 0](1422), float64(12), int64(1), object(12)
memory usage: 250.0+ MB
None


In [13]:
print(df)

                                                  acct_id  extenders  \
0       00000950dea4a869e9fe70d823444d418c5abebbd8e830...          0   
1       00002ec815678c6b9837ad4c9db82ac7fe5fb6af5dc229...          1   
2       000050825287b713415614a237757da4cd7517365cbce2...          1   
3       000060be4aa292815abc44ab6fe96015b89e83b21c8a63...          0   
4       00008c39885815e42a0bb750cee199cd4da741a5645705...          1   
...                                                   ...        ...   
789684  ffffcdb556e20f0bb172fd491886aceb9d01f963f80bd8...          0   
789685  ffffce839f7797215ce12aaf953e511b39f10b41f988f8...          1   
789686  ffffd57bcfde4896e5eb5bf6509a01284312777893c0bc...          0   
789687  ffffd68d1c03300303527bd26151f47e2ffbc637ba1e6e...          0   
789688  ffffe0b3d21258c06d013a20ec8d4c167f340087856583...          0   

        wireless_clients_count  wired_clients_count  rx_avg_bps  tx_avg_bps  \
0                    -1.092683            -0.638795    0

In [10]:
df.to_csv('cleaned_data.csv', index=False)